# Rasa Tutorial: The Basics

### Install Dependencies

In [2]:
!pip3 install git+https://github.com/mit-nlp/MITIE.git
!pip3 install --ignore-requires-python rasa-nlu
!pip3 install --ignore-requires-python rasa-core==0.14.5
!wget https://github.com/mit-nlp/MITIE/releases/download/v0.4/MITIE-models-v0.2.tar.bz2
!tar jxf MITIE-models-v0.2.tar.bz2

  Cloning https://github.com/mit-nlp/MITIE.git to /tmp/pip-req-build-5e4x7n2j
  Running command git clone --filter=blob:none --quiet https://github.com/mit-nlp/MITIE.git /tmp/pip-req-build-5e4x7n2j
  Resolved https://github.com/mit-nlp/MITIE.git to commit 42dbcd5dad55e9d5bc76580fc9582097342d9916
  Preparing metadata (setup.py) ... done
  Using cached rasa_nlu-0.15.1-py3-none-any.whl (147 kB)
  Using cached cloudpickle-0.6.1-py2.py3-none-any.whl (14 kB)
  Using cached gevent-1.5.0.tar.gz (5.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached klein-17.10.0-py2.py3-none-any.whl (30 kB)
  Using cached boto3-1.28.3-py3-none-any.whl (135 kB)
  Using cached packaging-18.0-py2.py3-none-any.whl (21 kB)
  Using cached future-0.17.1-py3-none-any.whl
  Using cached jsonschema-2.6.0-py2.py3-none-any.whl (39 kB)
  Using cached matplotlib-2.2.5-cp310-cp310-linux_x86_64.whl
  Using cached simplejson

# 1. Rasa NLU: Interpreters
## (User) Utterances -> (User) Intents & Entities

### 1.a. Configure Interpreter Pipeline

In [3]:
with open('interpreter.yml','w') as f:
  f.write('''
  language: "en"

  pipeline:
  - name: "nlp_mitie"
    model: "MITIE-models/english/total_word_feature_extractor.dat"
  - name: "tokenizer_mitie"
  - name: "ner_mitie"
  - name: "ner_synonyms"
  - name: "intent_entity_featurizer_regex"
  - name: "intent_featurizer_mitie"
  - name: "intent_classifier_sklearn"
  ''')

### 1.b. Prepare Interpreter Training Data

Format:

''##intent: intent_label"

"-example utterance for intent "

"-another example utterance with an [entity] (entity_label)"

In [4]:
with open('interpreter_dataset.md','w') as f:
  f.write('''

  ## intent:intent_bye
  - Bye
  - Goodbye
  - See you later
  - gotta go
  - See you
  - i'm off

  ## intent:intent_greet
  - Hi
  - Hey
  - Hello
  - Good morning
  - hi there
  - greetings

  ## intent:intent_thank
  - Thanks
  - Thank you
  - cheers
  - thanks a lot
  - cool thank you

  ## intent:intent_yes
  - yes
  - yeah
  - sure
  - absolutely
  - yep

  ## intent:intent_no
  - no
  - no way jose
  - nope
  - absolutely not
  - i don't think so

  ## intent:intent_name
  - My name is [Alice](name)
  - I'm [Lucy](name)
  - I am [Richard](name)
  - Call me [Sally](name)

  ## intent:intent_question
  - I have a question about [robots](topic)
  - Can i ask you something
  - What's [AI](topic) all about anyway
  - Can you help me understand what an [LSTM](topic) is?
  - I need to ask someone about something
  - can you help me understand [the difference between a CNN and an RNN](topic)?

  ''')

### 1.c. Train the Interpreter

In [5]:
!python3 -m rasa_nlu.train -c interpreter.yml -d interpreter_dataset.md -o models --fixed_model_name interpreter --verbose

/usr/bin/python3: Error while finding module specification for 'rasa_nlu.train' (ModuleNotFoundError: No module named 'rasa_nlu')


In [6]:
!pip install -r requirement.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirement.txt'


### 1.d. Test the Interpreter

In [7]:
from rasa_core.interpreter import RasaNLUInterpreter
interpreter = RasaNLUInterpreter('models/default/interpreter')

ModuleNotFoundError: ignored

In [8]:
interpreter.parse(input())

NameError: ignored

# 2. Rasa Core: Policies
## (User) Intents -> (Bot) Utterances & Actions

### 2.a. Configure Policy Pipeline


In [9]:
with open("policies.yml",'w') as f:
  f.write('''
    policies:
    - name: KerasPolicy
      epochs: 100
      max_history: 5
    - name: FallbackPolicy
      fallback_action_name: 'action_default_fallback'
    - name: MemoizationPolicy
      max_history: 5
    - name: FormPolicy
    - name: MappingPolicy
''')

### 2.b. Define the Bot's Utterances

In [10]:
with open('domain.yml','w') as f:
  f.write('''

  intents:
  - intent_greet
  - intent_bye
  - intent_thank
  - intent_yes
  - intent_no
  - intent_name
  - intent_question

  entities:
  - name
  - topic

  slots:
    name:
      type: text

  actions:
  - utter_name
  - utter_thank
  - utter_greet
  - utter_salaam
  - utter_bye
  - utter_sorry
  - utter_confusion
  - utter_expertNetwork

  templates:
    utter_name:
    - text: "What's your name?"
    - text: "My name is Ash. What can I call you?"

    utter_greet:
    - text: "Nice to you meet you {name}. How can I help?"
    - text: "Lovely to meet you {name}"
    - text: "Pleased to meet you {name}"
    - text: "Hi {name}. A pleasure meeting you"

    utter_salaam:
    - text: "Al-salamu 'alaykum {name}"
    - text: "Salaam {name}"
    - text: "Peace be upon you {name}"

    utter_bye:
    - text: "Talk to you later!"
    - text: "Bye"
    - text: "Good bye"
    - text: "Come back soon"

    utter_thank:
    - text: "My pleasure."
    - text: "Most welcome"
    - text: "Don't mention it"

    utter_sorry:
    - text: "I'm sorry"
    - text: "Sorry"
    - text: "forgive me"

    utter_confusion:
    - text: "I don't know"
    - text: "I'm not sure"
    - text: "me no understand"

    utter_expertNetwork:
    - text: "Interesting!  Lets see if someone in our network knows about {topic} ..."
      image: "https://s3.eu-west-2.amazonaws.com/cognitionx-assets/images/Cx_logo_Monogram_square.png"
    - text: "Let me connect you with our network of experts ..."
      image: "https://s3.eu-west-2.amazonaws.com/cognitionx-assets/images/Cx_logo_Monogram_square.png"
    - text: "sure, I'll connect you to an expert now ..."
      image: "https://s3.eu-west-2.amazonaws.com/cognitionx-assets/images/Cx_logo_Monogram_square.png"

  ''')

### 2.c Prepare Policy Training Data

Format:

"## story_name"

"* intent {"entity_label":"entity_value"}"

"   -   uterrance_or_action"

"   -   utterance_or_action"

In [11]:
with open('policy_dataset.md','w') as f:
  f.write('''

  ## story_greet
  * intent_greet
   - utter_greet
   - utter_name

  ## story_bye
  * intent_bye
   - utter_bye

  ## story_no
  * intent_no
   - utter_sorry

  ## story_thank
  * intent_thank OR intent_yes
   - utter_thank

  ## story_AIquestion
  * intent_question{"topic":"AI"}
   - utter_expertNetwork

  ## story_otherquestion
  * intent_question
   - utter_sorry
   - utter_confusion

  ## story_name
  * intent_name
   - utter_greet

  ## story_name
  * intent_name{"name":"Mohammed"}
   - utter_salaam

  ''')

### 2.d. Train the Policy

In [12]:
!python3 -m rasa_core.train -d domain.yml -s policy_dataset.md -o models/dialogue

/usr/bin/python3: Error while finding module specification for 'rasa_core.train' (ModuleNotFoundError: No module named 'rasa_core')


### 2.e Test the Policy

In [13]:
from rasa_core.agent import Agent
policy = Agent.load('models/dialogue', interpreter=interpreter)

ModuleNotFoundError: ignored

In [14]:
policy.handle_message(input())

NameError: ignored

# Let's chat with our Rasa Bot

In [15]:
DEFAULT_URLS = ["https://coubsecure-s.akamaihd.net/get/b54/p/coub/simple/cw_timeline_pic/7b2b17d8f24/15c695d0259a8b2231793/big_1518477107_image.jpg",
                "https://i0.wp.com/i.ytimg.com/vi/iqhKSgOKUlM/maxresdefault.jpg",
                "https://i.imgur.com/yX0HpCU.gif?noredirect"]
TEMPLATE_HTML = '''
<div class="chat-window"
<p>{}<p>
<img src = {} width="75" height="75"></img>
</div>
'''

import IPython,random

def chat():
  while True:
    user_utterance = input()
    responses = policy.handle_message(user_utterance)
    responses_html = ''.join([TEMPLATE_HTML.format('',response["text"].split("Image:")[-1]) if "Image:" in response["text"] else TEMPLATE_HTML.format(response["text"],random.choice(DEFAULT_URLS)) for response in responses])
    IPython.display.display(IPython.display.HTML(responses_html))
    if user_utterance == "bye":
      break

In [16]:
chat()

heloo


NameError: ignored